In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import flywheel 
fw = flywheel.Client() 
collection_id = '5eb5081448fe1b1e5792a7a9' 
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)] 

## What's on Flywheel?

In [2]:
groups = []
for ses in sessions:  
    group_id = ses.parents['group']  
    group = fw.get(group_id)  
    groups.append(group.label) 
print("Done")

Done


In [3]:
group_list = list(set(groups))
n_list = []
for g in group_list:
    n_list.append(groups.count(g))
    print("{}: {}".format(g, groups.count(g)))

Detre Group: 62
Penn FTD Center: 435
Oathes Lab: 95
David Wolk Lab: 210
Davis Group: 16
Connectome: 43


### Checking Oathes lab discrepancy 94/95

In [ ]:
oathes = []
for ses in sessions:
    group_id = ses.parents['group']
    group = fw.get(group_id)
    if group.label == 'Oathes Lab':
        sub = fw.get(ses.parents['subject'])
        oathes.append(sub.label)
print('Done')        

In [ ]:
print(len(oathes))

In [ ]:
old_oathes_df = pd.read_csv('uploaded/zap_metadata.csv')
old_oathes_list = old_oathes_df['Record ID']
print(len(old_oathes_list))

Now we remove the patients, keeping only the controls.

In [ ]:
just_controls_df = old_oathes_df[old_oathes_df['Record ID'].str.startswith('C')]
just_controls_list = list(just_controls_df['Record ID'])
print(len(just_controls_list))

Subtract sets to find the discrepancy:

In [ ]:
set(oathes) - set(just_controls_list)

# Calculate metadata
_______________________________________________________________________________________________


#### Setup

In [ ]:
# group_list = ['Aguirre', 'Davis', 'Detre', 'Oathes', 'Wolk']
#group_list = ['Davis Group', 'David Wolk Lab', 'Connectome', 'Detre Group', 'Oathes Lab', 'FTDC']
df = pd.read_csv('all_subs.csv')

### Calculate % Female for each group

In [ ]:
sex_dict = {}
for group in group_list:
    group_df = df[df['Group'] == group]
    group_sex_series = group_df['Sex']
    breakdown = group_sex_series.value_counts()
    # calculate
    n_fems = breakdown['Female']
    n_total = len(group_sex_series)
    sex_dict[group] = (n_fems/n_total)*100

sex_df = pd.DataFrame.from_dict(sex_dict, orient='index', columns=['% Female'])

### Calculate mean and sd of age for each group

In [ ]:
mean_dict = {}
for group in group_list:
    group_df = df[df['Group'] == group]
    group_age_series = group_df['Age']
    #calculate
    group_age_series.dropna(inplace=True)
    mean = np.mean(group_age_series.to_numpy(dtype='float32'))
    sd = np.std(group_age_series.to_numpy(dtype='float32'))
    values = [mean, sd]
    mean_dict[group] = values
    
mean_df = pd.DataFrame.from_dict(mean_dict, orient='index', columns = ['mean','sd'])

In [ ]:
n_list = [16, 210, 43, 62, 95, 459]


In [ ]:
big_df = mean_df
big_df['% Female'] = sex_df['% Female']
big_df['N'] = n_list
big_df.to_csv('summary.csv')

# Plot histogram
_______________________________________________________________________________________________


In [ ]:
df_noftdc = df[df.Group != 'FTDC']

In [ ]:
sns.distplot(df["Age"], norm_hist=False, bins=50, kde=False)
sns.distplot(df_noftdc["Age"], norm_hist=False, bins=50, kde=False)